In [1]:
# IMPORT STUFF
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
import math
from collections import Counter
import json
import os
import ast
import multiprocessing

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# SET SOME GLOBAL VARIABLES
QUERIES_PATH = "/kaggle/input/dis2023-project1-data/queries.jsonl"
CORPUS_PATH = "/kaggle/input/dis2023-project1-data/corpus.jsonl"
SAVEFILE_PATH = '/kaggle/input/savefile-txt/savefile.txt'
SAVEFILE_QUERIES_PATH = '/kaggle/input/savefile-queries-txt/savefile_queries.txt'
RESULTS_PATH = '/kaggle/working/submission.csv'

In [3]:
%%time
# --------- PREPROCESSING START ---------
""" Commented as we have preprocessed files as part of the input

# TOKENIZE CORPUS TEXTS AND SAVE TO SAVE-FILE
if not os.path.exists(SAVEFILE_PATH):
    with open(CORPUS_PATH) as f:
        raw_corpus = f.readlines()
        corpus = [json.loads(q) for q in raw_corpus]
        
    stemmer = PorterStemmer() 
    trans = str.maketrans(string.punctuation, " " * len(string.punctuation))

    f = open(SAVEFILE_PATH, "a")
    def tokenizeAndSave(nr, text):
        text = text.strip().translate(trans)
        tokens = nltk.word_tokenize(text)
        stemmed = [stemmer.stem(word.lower()) for word in tokens if word not in stopwords.words('english')]
        f.write(str(nr) + "," + ' '.join(stemmed) + "\n")

    for c in corpus:
        tokenizeAndSave(int(c["_id"]), c["text"])
    f.close()

# TOKENIZE QUERIES AND SAVE TO SAVE-FILE
if not os.path.exists(SAVEFILE_QUERIES_PATH):
    queries = dict()
    with open(QUERIES_PATH) as f:
        raw_queries = f.readlines()
        for q in raw_queries:
            obj = json.loads(q)
            queries[int(obj["_id"])] = obj["text"]
            
    stemmer = PorterStemmer() 
    trans = str.maketrans(string.punctuation, " " * len(string.punctuation))
    
    f = open(SAVEFILE_QUERIES_PATH, "a")
    def tokenizeAndSave(nr, text):
        text = text.strip().translate(trans)
        tokens = nltk.word_tokenize(text)
        stemmed = [stemmer.stem(word.lower()) for word in tokens if word not in stopwords.words('english')]
        f.write(str(nr) + "," + ' '.join(stemmed) + "\n")

    for q in queries.keys():
        tokenizeAndSave(q, queries[q])
    f.close()
"""
# --------- PREPROCESSING END ---------

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


' Commented as we have preprocessed files as part of the input\n\n# TOKENIZE CORPUS TEXTS AND SAVE TO SAVE-FILE\nif not os.path.exists(SAVEFILE_PATH):\n    with open(CORPUS_PATH) as f:\n        raw_corpus = f.readlines()\n        corpus = [json.loads(q) for q in raw_corpus]\n        \n    stemmer = PorterStemmer() \n    trans = str.maketrans(string.punctuation, " " * len(string.punctuation))\n\n    f = open(SAVEFILE_PATH, "a")\n    def tokenizeAndSave(nr, text):\n        text = text.strip().translate(trans)\n        tokens = nltk.word_tokenize(text)\n        stemmed = [stemmer.stem(word.lower()) for word in tokens if word not in stopwords.words(\'english\')]\n        f.write(str(nr) + "," + \' \'.join(stemmed) + "\n")\n\n    for c in corpus:\n        tokenizeAndSave(int(c["_id"]), c["text"])\n    f.close()\n\n# TOKENIZE QUERIES AND SAVE TO SAVE-FILE\nif not os.path.exists(SAVEFILE_QUERIES_PATH):\n    queries = dict()\n    with open(QUERIES_PATH) as f:\n        raw_queries = f.readlines

In [4]:
%%time
# --------- LOADING DATA START ---------
# LOAD TEXTS FROM FILES
# load documents
documents = {}
with open(SAVEFILE_PATH) as f:
    raw = f.readlines()
    for line in raw:
        nr, txt = line.split(',')
        documents[int(nr)] = txt.split()

# load queries
queries = {}
with open(SAVEFILE_QUERIES_PATH) as f:
    raw = f.readlines()
    for line in raw:
        nr, txt = line.split(',')
        queries[int(nr)] = txt.split()

# TRAIN- AND TEST-FILES
TASK1_TEST = '/kaggle/input/dis-project-1-text-retrieval/task1_test.tsv'
TASK1_TRAIN = '/kaggle/input/dis-project-1-text-retrieval/task1_train.tsv'
TASK2_TEST = '/kaggle/input/dis-project-1-text-retrieval/task2_test.tsv'
TASK2_TRAIN = '/kaggle/input/dis-project-1-text-retrieval/task2_train.tsv'

# read in train-data of task1
tb = pd.read_table(TASK1_TRAIN)
ind_gen = list(tb.iterrows())
task1_traindata = {}
for _, v in ind_gen:
    task1_traindata[int(v['query-id'])] = int(v['corpus-id'])
    
# read in test-data of task1
tb = pd.read_table(TASK1_TEST)
ind_gen = list(tb.iterrows())
task1_tests = []
for _, v in ind_gen:
    task1_tests.append((int(v['id']), int(v['query-id'])))

# read in train-data of task2
tb = pd.read_table(TASK2_TRAIN)
ind_gen = list(tb.iterrows())
task2_traindata = {}
for _, v in ind_gen:
    task2_traindata[int(v['query-id'])] = (ast.literal_eval(v['corpus-id']), ast.literal_eval(v['score']))

# read in test-data of task2
tb = pd.read_table(TASK2_TEST)
ind_gen = list(tb.iterrows())
task2_tests = []
for _, v in ind_gen:
    task2_tests.append((int(v['id']), int(v['query-id']), ast.literal_eval(v['corpus-id'])))
# --------- LOADING DATA END ---------

CPU times: user 1min 28s, sys: 7.93 s, total: 1min 36s
Wall time: 1min 39s


In [5]:
# Open results file
f = open(RESULTS_PATH, "w")
f.write('id,corpus-id,score\n')

19

In [6]:
%%time
# --------- COMPUTATION TASK 1 START ---------
# create the inverted index
inv_idx = {}
for nr in documents.keys():
    for word in set(documents[nr]):
        if word in inv_idx:
            inv_idx[word].append(nr)
        else:
            inv_idx[word] = [nr]
    
# use inverted index to get n "most important" documents from corpus
def doc_full_ranking(query, n):
    indices = []
    for word in query:
        if word in inv_idx:
            indices += inv_idx[word]
    return [nr for nr,_ in Counter(indices).most_common(n)]

# create a vocabulary from some texts
def voc_from_txts(texts):
    voc = set()
    voc.update(*texts)
    voc = list(voc)
    return voc

# calculate idf-values for all terms in a vocabulary
def idf_values(voc, txts):
    idf = {}
    num_txts = len(txts)
    for term in voc:
        idf[term] = num_txts/sum(term in txt for txt in txts)
    return idf

# vectorize documents in "doc" list based on TF-IDF metric
def vectorize(doc, voc, idf):
    counts = Counter(doc)
    max_count = counts.most_common(1)[0][1]
    
    vector = [0]*len(voc)
    for i,term in enumerate(voc):
        c = counts[term]
        if c != 0:
            vector[i] = idf[term] * c/max_count
    return vector

# calculate cosine simliarity between two vectors
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

# classic Vector Retrieval based on TF-IDF metric
def vector_retrieval(query, doc_ids, n):
    docs = [documents[did] for did in doc_ids]
    voc = voc_from_txts(docs)
    idfs = idf_values(voc, docs)
    
    vecs = [vectorize(s, voc, idfs) for s in docs]
    query_vec = vectorize(query, voc, idfs)
    
    sims = [(d, cosine_similarity(query_vec, v)) for d, v in zip(doc_ids, vecs)]
    sims = sorted(sims, key=lambda item: -item[1])
    return [d for d,_ in sims[:n]]

# main retrieval for a single query
def retrieval(qid):
    n_small = 10
    n_big = 70
    assert n_small <= n_big
    query = queries[qid]
    # Step 1: use inverted index
    extracted_docs = doc_full_ranking(query, n_big)
    # Step 2: use vector retrieval on documents extracted in phase 1
    extracted_docs = vector_retrieval(query, extracted_docs, n_small)
    return extracted_docs
    
"""# Training Data
PROCESSES = 4
keys = list(task1_traindata.keys())[:100]
params = []
for k in keys:
    params.append((k,))
with multiprocessing.Pool(PROCESSES) as pool:
    results = [pool.apply_async(retrieval, p) for p in params]

    correct = 0
    for i, r in enumerate(results):
        extracted_docs = r.get()
        solution = task1_traindata[keys[i]]
        if solution in extracted_docs:
            correct += 1
        if (i+1) % 10 == 0:
            print((i+1), correct, "->", (correct / (i + 1)) * 100, "%")
"""

# Testing Data
PROCESSES = 4
params = []
for i,qid in task1_tests:
    params.append((i,(qid,)))
with multiprocessing.Pool(PROCESSES) as pool:
    results = [(i, pool.apply_async(retrieval, p)) for i,p in params]
    for i, r in results:
        extracted_docs = r.get()
        # write to results file
        f.write(f"{i},\"{extracted_docs}\",-1\n")
        if i % 500 == 0:
            print(i)
# --------- COMPUTATION TASK 1 END ---------

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
CPU times: user 1min 7s, sys: 3.06 s, total: 1min 10s
Wall time: 9min 4s


In [7]:
# --------- COMPUTATION TASK 2 START ---------
# map cosine-similarities to the set {0, 1, 2, 3} 
def float_to_rank(f):
    if f > .1:
        return 3
    if f > .05:
        return 2
    if f > .01:
        return 1
    return 0

# re-rank documents based on a query
def topk_reranking(qid, corpusids):
    reranking_docs = [documents[cid] for cid in corpusids]
    reranking_voc = voc_from_txts(reranking_docs)
    reranking_idf = idf_values(reranking_voc, reranking_docs)
    
    reranking_vecs = [vectorize(s, reranking_voc, reranking_idf) for s in reranking_docs]
    query_vec = vectorize(queries[qid], reranking_voc, reranking_idf)
    
    sims = [float_to_rank(cosine_similarity(query_vec, v)) for v in reranking_vecs]
    return sims

"""# Training Data
avg = 0
for qid in task2_traindata.keys():
    corpusids, scores = task2_traindata[qid]
    res = topk_reranking(qid, corpusids)
    # TODO: somehow this gives a completely wrong result :(
    how_good = sum(a == b for a,b in zip(scores, res)) / len(scores)
    avg += how_good
# note: this gets an average of around 42%. if we just always return 0 we get >60% :(
print("Average: ", (avg / len(task2_traindata.keys())) * 100, "%")"""

# Testing Data
for i, qid, corpusids in task2_tests:
    res = topk_reranking(qid, corpusids)
    f.write(f"{i},-1,\"{res}\"\n")
# --------- COMPUTATION TASK 2 END ---------

In [8]:
# Close results file
f.close()